# Elhub API data - Gridloss - Summerproject 2025

## Merging data from Elhub API

Bjørn Eirik Rognskog Nordbak

### Importing data from Elhub API
https://api.elhub.no/energy-data-api#/grid-areas

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

oslo = ZoneInfo("Europe/Oslo")

def fetch_window(start_dt, end_dt):
    params = {
        "dataset":   "LOSS_PER_MGA_HOUR",
        "startDate": start_dt.isoformat(),
        "endDate":   end_dt.isoformat(),
    }
    url = "https://api.elhub.no/energy-data/v0/grid-areas"
    resp = requests.get(url, params=params)
    obj = resp.json()
    
    # --- safeguard: if there's no "data", bail with empty DF ----
    raw = obj.get("data")
    if raw is None:
        print(f"  → no 'data' for {start_dt.date()} → {end_dt.date()}, skipping")
        return pd.DataFrame()
    
    # otherwise flatten
    df = pd.json_normalize(
        raw,
        record_path=["attributes", "lossPerMgaHour"],
        meta=[
            ["attributes", "eic"],
            ["attributes", "name"],
            ["attributes", "status"],
        ],
        errors="ignore"
    ).rename(columns={
        "attributes.eic":    "eic",
        "attributes.name":   "name",
        "attributes.status": "status",
    })
    return df

# loop as before
span_start = datetime(2023,1,1,0,0, tzinfo=oslo)
span_end   = datetime(2025,6,1,0,0, tzinfo=oslo)
window = timedelta(days=7)

all_chunks = []
cur = span_start
while cur < span_end:
    nxt = min(cur + window, span_end)
    print(f"Fetching {cur.date()} → {nxt.date()}")
    dfc = fetch_window(cur, nxt)
    all_chunks.append(dfc)
    cur = nxt

big_df = pd.concat(all_chunks, ignore_index=True)


### Save the data to a CSV file (optional)

In [ ]:
import pandas as pd

# 1. Save your DataFrame to CSV
# Replace big_df with your DataFrame variable
big_df.to_csv('big_df.csv', index=False)

### Reading data from premade CSV file (if neede)

In [1]:
import pandas as pd

big_df = pd.read_csv('big_df.csv')

big_df.head()

,calculatedLossFraction,calculatedLossQuantityKwh,endTime,gridArea,lastUpdatedTime,netInfeedQuantityKwh,priceArea,startTime,eic,name,status
0,0.062064,1451.970,2023-01-01T01:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23394.820,NO2,2023-01-01T00:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
1,0.059934,1399.627,2023-01-01T02:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23352.914,NO2,2023-01-01T01:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
2,0.061977,1430.195,2023-01-01T03:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23076.060,NO2,2023-01-01T02:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
3,0.060450,1373.440,2023-01-01T04:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,22720.318,NO2,2023-01-01T03:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
4,0.060322,1360.366,2023-01-01T05:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,22551.768,NO2,2023-01-01T04:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active


### Importing data from Elhub API
https://api.elhub.no/energy-data/v0/parties

In [3]:
import requests
import pandas as pd

# 1) Fetch the JSON
url = "https://api.elhub.no/energy-data/v0/parties"
resp = requests.get(url, headers={"Accept":"application/vnd.api+json"})
resp.raise_for_status()
j = resp.json()

# 2) Grab the list under "data"
items = j["data"]

# 3) Normalize into a DataFrame
#    This will produce columns: id, type, attributes.gln, attributes.name, attributes.organisationNumber, attributes.partyTypes
df = pd.json_normalize(items, sep="_")

# 4) (Optional) rename columns to something friendlier
df.columns = (
    df.columns
      .str.replace("attributes_", "")
      .str.replace("partyTypes", "party_types")
)
df


,id,type,gln,name,organisationNumber,party_types,relationships_grid-area_data
0,7080003816870,parties,7080003816870,Linja AS Regulert kraftleverandør,912631532,"[BalanceResponsible, RegulatedBalanceSupplier]",[]
1,7080003819864,parties,7080003819864,Linja (tidl SFE) REG Kraftleverandør,984882114,[RegulatedBalanceSupplier],[]
2,7080003823953,parties,7080003823953,Krødsherad Everk KF - Regulert kraftleverandør,971030658,[RegulatedBalanceSupplier],[]
3,7080005052382,parties,7080005052382,Telenor Eiendom Holding As,971050365,[BalanceSupplier],[]
4,7080003819383,parties,7080003819383,Barents Nett AS Pliktleveranse,971058854,[RegulatedBalanceSupplier],[]
...,...,...,...,...,...,...,...
881,7080003400673,parties,7080003400673,NECS,962986633,[CertificateIssuer],[]
882,7080004073456,parties,7080004073456,Statnett Forskningsprosjekt 2019-2021,962986633,[ThirdParty],[]
883,7080004066618,parties,7080004066618,Nettselskapet AS reg. kraftlev,921688679,"[ServiceProvider, RegulatedBalanceSupplier]",[]
884,7080004066526,parties,7080004066526,Nettselskapet AS - Sodvin-Regulert kraftlev,921688679,"[ServiceProvider, RegulatedBalanceSupplier]",[]
